In [ ]:
#Verificar si tienes GPU para mayor velocidad de entrenamiento
!nvidia-smi

In [ ]:
#####
# 1 #
#####
# INSTALAR DEPENDENCIAS DE PIPER #
!apt-get update
!apt-get install -y --fix-missing git unzip wget
%cd /kaggle/working
!git clone -q https://github.com/rmcpantoja/piper
%cd /kaggle/working/piper/src/python
!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"
!rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info
!rm -rf /opt/conda/lib/python3.10/site-packages/numpy-1.26.4.dist-info
!pip install numpy gdown
!pip install openai-whisper
#!pip install -q -r requirements.txt
!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy==1.24 onnxruntime>=1.11.0 pytorch-lightning==1.7.7 torch==1.13.0+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install -q torchtext==0.14.0 torchvision==0.14.0
!python3 -m pip install --no-binary numpy numpy==1.21.5
!python3 -m pip install --force-reinstall numpy
#!pip install -U openai-whisper
# fixing recent compativility issues:
!pip install -q torchaudio==0.13.0 torchmetrics==0.11.4 faster_whisper
!bash build_monotonic_align.sh
# Useful vars:
use_whisper = True
print("Done!")

In [ ]:
#####
# 2 #
#####
# No usar si ya importaste tu dataset anteriormente
import os
import csv
import re

# Función para extraer el ID de Google Drive del enlace
def extraer_id_de_enlace(enlace):
    patron = r'/file/d/([a-zA-Z0-9_-]+)'
    coincidencia = re.search(patron, enlace)
    if coincidencia:
        return coincidencia.group(1)
    else:
        return None
import whisper

# Variables para los enlaces de Google Drive
# Reemplazar el link de wavs por el suyo
wavs_link = "https://drive.google.com/file/d/XXXXXXXXXXXXXXXXXXXXXXXXX/view?usp=drive_link"
metadata_link = None  # Cambia esto si tienes un enlace de metadata

# Extraer los IDs de los enlaces
wavs_id = extraer_id_de_enlace(wavs_link)
metadata_id = extraer_id_de_enlace(metadata_link) if metadata_link else None

# Verificar si wavs_id está especificado
if not wavs_id:
    print("Error: No se ha especificado el ID de descarga del zip con wavs.")
    exit(1)

# Crear el directorio si no existe
dataset_dir = "/kaggle/working/piper/my-dataset"
wavs_dir = os.path.join(dataset_dir, "wavs")

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

# Verificar si los archivos ya existen en el equipo y eliminarlos si es así
if os.path.exists(os.path.join(wavs_dir, "wavs.zip")):
    os.remove(os.path.join(wavs_dir, "wavs.zip"))
if os.path.exists(os.path.join(dataset_dir, "metadata.csv")):
    os.remove(os.path.join(dataset_dir, "metadata.csv"))

# Descargar los archivos usando los IDs proporcionados
!mkdir -p /kaggle/working/piper/my-dataset/wavs
!gdown --id {wavs_id} -O {wavs_dir}/wavs.zip

if metadata_id:
    !gdown --id {metadata_id} -O {dataset_dir}/metadata.csv

# Descomprimir los archivos y forzar a sobrescribir sin pedir confirmación
os.chdir(wavs_dir)
!unzip -o wavs.zip
# Eliminar el archivo zip
os.remove("wavs.zip")

# Iniciar la transcripción y guardar en el archivo CSV si metadata_id no está especificado
if not metadata_id:
    # Definir la carpeta donde están los archivos de audio
    carpeta_audio = wavs_dir

    # Listar los archivos de la carpeta, filtrar solo archivos .wav y ordenar por nombre
    archivos_audio = sorted([archivo for archivo in os.listdir(carpeta_audio) if archivo.endswith('.wav')], key=lambda x: int(x.replace('.wav', '')))

    # Abrir el archivo CSV en modo de escritura
    with open(os.path.join(dataset_dir, 'metadata.csv'), 'w', newline='', encoding='utf-8') as archivo_csv:
        # Crear un escritor CSV
        escritor_csv = csv.writer(archivo_csv, delimiter='|')

        # Cargar el modelo Whisper (puedes cambiar "large" por "small" o "medium" según tus necesidades)
        model = whisper.load_model("large")

        # Iterar sobre los archivos de audio
        for i, archivo in enumerate(archivos_audio, start=1):
            # Ruta completa del archivo de audio
            ruta_archivo = os.path.join(carpeta_audio, archivo)

            # Transcribir el archivo de audio
            result = model.transcribe(ruta_archivo)

            # Extraer el texto de la transcripción
            texto_transcrito = result["text"]

            # Crear el nombre del archivo con el número y extensión correspondiente
            nombre_archivo = f"{i}.wav"

            # Formato de salida (nombre_archivo|texto_transcrito)
            formato_salida = f"{nombre_archivo}|{texto_transcrito}"

            # Imprimir el formato solicitado en el log de la consola
            print(formato_salida)

            # Escribir la transcripción en el archivo CSV
            escritor_csv.writerow([nombre_archivo, texto_transcrito])
else:
    print("La transcripción se ha omitido porque metadata_id está especificado.")

In [ ]:
%cd /kaggle/working/piper/my-training/lightning_logs/version_1/checkpoints/
!rm 'epoch=10375-step=1755104.ckpt'

In [ ]:
#####
# 3 #
#####
# PREPARAR DATASET
#Lang code list: https://github.com/espeak-ng/espeak-ng/blob/master/docs/languages.md
%cd /kaggle/working/piper/src/python
!python3 -m piper_train.preprocess \
  --language es-419 \
  --input-dir /kaggle/working/piper/my-dataset \
  --output-dir /kaggle/working/piper/my-training \
  --dataset-format ljspeech \
  --single-speaker \
  --sample-rate 22050

In [ ]:
#####
# 4 #
#####
############################################
# EMPEZAR A ENTRENAR/CONTINUAR ENTRENANDO #
###########################################
# EMPIEZA A ENTRENAR TU DATASET CON EL MODELO BASE O SI ENCUENTRA 
# UN PUNTO DE ENTRENAMIENTO ACTUAL SIGUE CON EL ENTRENAMIENTO
%cd /kaggle/working/piper/src/python
import os
import shutil
import requests

def listar_carpetas_version(ruta):
    version_folders = []
    for root, dirs, files in os.walk(ruta):
        for dir in dirs:
            if dir.startswith("version_"):
                version_folders.append(os.path.join(root, dir))
    return version_folders

# Ejemplo de uso
piper_logs = "/kaggle/working/piper/my-training/lightning_logs/"
version_folders = listar_carpetas_version(piper_logs)

# Almacenar nombres de carpetas que contienen la carpeta "checkpoints"
checkpoints_folders = []

# Buscar las carpetas dentro de piper_logs
for version_folder in version_folders:
    folder_path = os.path.join(piper_logs, version_folder)
    if os.path.exists(folder_path):
        checkpoints_path = os.path.join(folder_path, "checkpoints")
        # Verificar si existe la carpeta "checkpoints"
        if os.path.exists(checkpoints_path):
            checkpoints_folders.append(checkpoints_path)

# Almacenar el archivo .ckpt con el menor valor en epoch
selected_checkpoint = None
min_epoch = float('inf')

# Revisar las carpetas que contienen la carpeta "checkpoints"
for checkpoints_folder in checkpoints_folders:
    # Obtener lista de archivos en la carpeta "checkpoints"
    files = os.listdir(checkpoints_folder)
    # Filtrar archivos con el formato epoch=XXXXX-step=XXXXX.ckpt
    ckpt_files = [file for file in files if file.endswith(".ckpt")]
    # Verificar si hay archivos .ckpt
    if ckpt_files:
        for ckpt_file in ckpt_files:
            # Obtener el valor de epoch del archivo .ckpt
            epoch = int(ckpt_file.split("=")[1].split("-")[0])
            # Actualizar el archivo seleccionado si encontramos uno con menor valor en epoch
            if epoch < min_epoch:
                min_epoch = epoch
                selected_checkpoint = os.path.join(checkpoints_folder, ckpt_file)

# Si no se encontró un archivo .ckpt, descargar el archivo especificado
if selected_checkpoint is None:
    print("No se encontraron archivos .ckpt. Descargando el archivo de respaldo...")
    # Reemplaza el valor de backup_checkpoint_url en caso de querer usar otro punto de control
    backup_checkpoint_url = "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/es/es_ES/davefx/medium/epoch%3D5629-step%3D1605020.ckpt"
    response = requests.get(backup_checkpoint_url)
    if response.status_code == 200:
        # Guardar el archivo descargado en la carpeta de destino
        destination_folder = "/kaggle/working/piper/"
        os.makedirs(destination_folder, exist_ok=True)
        destination_file = os.path.join(destination_folder, "last.ckpt")
        with open(destination_file, "wb") as f:
            f.write(response.content)
        selected_checkpoint = destination_file
    else:
        print("Error al descargar el archivo de respaldo.")
else:
    print("Se encontró un archivo .ckpt.")

# Mostrar la ubicación del archivo .ckpt encontrado
print(f"Ubicación del archivo .ckpt encontrado: {selected_checkpoint}")

# Incluir la ubicación del archivo seleccionado en el comando de entrenamiento
full_checkpoint_path = selected_checkpoint
script_content = f"""
# Ejecutar el comando piper_train con el �ltimo checkpoint
python3 -m piper_train \\
    --dataset-dir /kaggle/working/piper/my-training \\
    --accelerator 'gpu' \\
    --devices 1 \\
    --batch-size 12 \\
    --validation-split 0.0 \\
    --num-test-examples 0 \\
    --max_epochs 20000 \\
    --resume_from_checkpoint "{full_checkpoint_path}" \\
    --checkpoint-epochs 5 \\
    --precision 32
"""

# Guardar el script en un archivo temporal
script_file = "/kaggle/working/piper/src/python/run_piper_train.sh"
with open(script_file, "w") as f:
    f.write(script_content)

print("Script creado exitosamente.")

# Ejecutar el script
!bash {script_file}  # Descomenta esta línea para ejecutar el script

/kaggle/working/piper/src/python
Se encontró un archivo .ckpt.
Ubicación del archivo .ckpt encontrado: /kaggle/working/piper/my-training/lightning_logs/version_3/checkpoints/epoch=6129-step=1639020.ckpt
Script creado exitosamente.
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:52: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v1.7. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:731: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  ckpt_path = ckpt_path or self.resume_from_checkpoint
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:345: UserWarning: The dirpath has changed from '/k

# Cargar modelo a Huggingface.co
![Huggingface](https://i.ibb.co/hRv6DCg/datasets-logo.png)

In [1]:
################################################
## SUBIR TU RESPALDO A TU REPO DE HUGGINGFACE ##
################################################
import os
import re
from huggingface_hub import HfApi

# Directorio base
base_dir = "/kaggle/working/piper/my-training/lightning_logs"

# Patrón para los checkpoints
ckpt_pattern = re.compile(r"epoch=(\d+)-step=(\d+)\.ckpt")

# Identificador del repositorio
# Crear repo tipo modelo para subir tu progreso
repo_id = "Usuario/RepoName"

# Token de acceso de HuggingFace
token = "TU_TOKEN_AQUI"

# Busca las carpetas version_X
version_folders = [folder for folder in os.listdir(base_dir) if folder.startswith("version")]

for version in version_folders:
    checkpoints_dir = os.path.join(base_dir, version, "checkpoints")
    if os.path.exists(checkpoints_dir):
        # Busca todos los archivos .ckpt
        ckpt_files = [file for file in os.listdir(checkpoints_dir) if file.endswith(".ckpt")]
        # Lista para almacenar los datos de los checkpoints
        ckpt_data = []
        for file in ckpt_files:
            match = ckpt_pattern.match(file)
            if match:
                epoch, step = map(int, match.groups())
                ckpt_data.append((epoch, step, file))
        # Si existen checkpoints, selecciona el que tenga menor epoch
        if ckpt_data:
            ckpt_data.sort(key=lambda x: x[0])
            _, _, ckpt_file = ckpt_data[0]
            # Path completo del archivo de checkpoint
            ckpt_path = os.path.join(checkpoints_dir, ckpt_file)
            api = HfApi()
            api.upload_file(
                path_or_fileobj=ckpt_path,
                path_in_repo=ckpt_file,
                repo_id=repo_id,
                repo_type="model",
                token=token
            )

# Subir también el archivo config.json
config_path = "/kaggle/working/piper/my-training/config.json"
api = HfApi()
api.upload_file(
    path_or_fileobj=config_path,
    path_in_repo="config.json",
    repo_id=repo_id,
    repo_type="model",
    token=token
)

epoch=6494-step=1663840.ckpt:   0%|          | 0.00/846M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HirCoir/piper-sorah-neuronal/commit/973728a49e269b2779f501ab01841871c0c36c75', commit_message='Upload config.json with huggingface_hub', commit_description='', oid='973728a49e269b2779f501ab01841871c0c36c75', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
######################################################################################
## GENERAR LINKS DE DESCARGA DE LOS ARCHIVOS CKPT Y JSON PARA PROBARLOS DESDE COLAB ##
######################################################################################

import os
import re

# Directorio base
base_dir = "/kaggle/working/piper/my-training/lightning_logs"

# Patrón para los checkpoints
ckpt_pattern = re.compile(r"epoch=(\d+)-step=(\d+)\.ckpt")

# Función para obtener la ruta completa del último archivo de checkpoint :)
def get_latest_checkpoint():
    # Busca las carpetas version_X
    version_folders = [folder for folder in os.listdir(base_dir) if folder.startswith("version")]
    latest_ckpt_path = None
    
    for version in version_folders:
        checkpoints_dir = os.path.join(base_dir, version, "checkpoints")
        if os.path.exists(checkpoints_dir):
            # Busca todos los archivos .ckpt
            ckpt_files = [file for file in os.listdir(checkpoints_dir) if file.endswith(".ckpt")]
            ckpt_data = []
            for file in ckpt_files:
                match = ckpt_pattern.match(file)
                if match:
                    epoch, step = map(int, match.groups())
                    ckpt_data.append((epoch, step, file))
            # Si existen checkpoints, selecciona el que tenga menor epoch
            # Normalmente si hay dos archivos el mas reciente está dañado y por ende toma el mas "antiguo"
            if ckpt_data:
                ckpt_data.sort(key=lambda x: x[0])
                _, _, ckpt_file = ckpt_data[0]
                latest_ckpt_path = os.path.join(checkpoints_dir, ckpt_file)
                break
    
    return latest_ckpt_path

# Función para obtener la ruta completa de config.json
def get_config_json_path():
    config_path = "/kaggle/working/piper/my-training/config.json"
    return config_path
import subprocess

# Función para subir un archivo a bashupload.com y obtener la URL de descarga :)
def upload_to_bashupload(file_path):
    result = subprocess.run(
        ['curl', '-s', 'https://bashupload.com/', '-T', file_path],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return result.stdout.strip()
    else:
        print(f"Error al subir {file_path}: {result.stderr}")
        return None

# Obtener la ruta del último checkpoint
latest_ckpt_path = get_latest_checkpoint()

# Subir el último checkpoint a bashupload.com
if latest_ckpt_path:
    checkpoint_download_url = upload_to_bashupload(latest_ckpt_path)
    if checkpoint_download_url:
        print(f"Uploaded checkpoint. Download URL: {checkpoint_download_url}")

# Obtener la ruta de config.json
config_path = get_config_json_path()

# Subir config.json a bashupload.com
if config_path:
    config_download_url = upload_to_bashupload(config_path)
    if config_download_url:
        print(f"Uploaded config.json. Download URL: {config_download_url}")

Uploaded checkpoint. Download URL: https://bashupload.com/FaNml/5S058.ckpt
Uploaded config.json. Download URL: https://bashupload.com/Kox49/config.json


# [Cuaderno de Inference Colab para probar modelos (No requiere CPU)](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_inference_(ckpt).ipynb)
# [Exportar modelos a .onnx para usarlos en cualquier sistema operativo, Windows, Linux, etc.](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_model_exporter.ipynb)
# [Exportar modelos a .onnx usando contenedores de HuggingFace (Solo funciona para checkpoints cargados a HuggingFace desde este cuaderno).](https://huggingface.co/spaces/HirCoir/Piper-onnx-export/blob/main/Dockerfile)